<a href="https://colab.research.google.com/github/IshantWadhwa4/Machine_Learning/blob/master/Supervisedlearning/MLEndtoEnd/MLEndtoEndPart1(Titanic_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML End to End 

## Step 1: Get Data

In [0]:
import pandas as pd

In [0]:
titanic_data = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-1/master/Data/Casestudy/titanic_train.csv')

In [3]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Step 2: Clean data

In [0]:
titanic_data.Embarked.fillna(titanic_data['Embarked'].mode()[0],inplace=True)
titanic_data.Age.fillna(titanic_data.Age.median(),inplace=True)
titanic_data.drop('Cabin',axis=1,inplace=True)
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch']+1
titanic_data['GenderClass'] = titanic_data.apply(lambda x: 'child' if x['Age'] < 15 else x['Sex'],axis=1)
titanic_data = pd.get_dummies(titanic_data, columns=['GenderClass','Embarked'], drop_first=True)
titanic = titanic_data.drop(['Name','Ticket','Sex','SibSp','Parch','PassengerId'], axis = 1)

In [6]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived              891 non-null int64
Pclass                891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
GenderClass_female    891 non-null uint8
GenderClass_male      891 non-null uint8
Embarked_Q            891 non-null uint8
Embarked_S            891 non-null uint8
dtypes: float64(2), int64(3), uint8(4)
memory usage: 38.4 KB


## Step 3: Apply Models

In [0]:
## convert data to train and text
X = titanic.drop('Survived',axis=1)
y = titanic['Survived']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,test_size=0.30)

In [0]:
# import all models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection



MLA = [
       
       LogisticRegression(random_state = 1),
       DecisionTreeClassifier(random_state = 1),
       RandomForestClassifier(random_state = 1)
       
       ]

#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .3, train_size = .7, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time', 'TrainTestDifference']
MLA_compare = pd.DataFrame(columns = MLA_columns)

MLA_Predict = titanic['Survived']




In [0]:
row_index = 0

for alg in MLA:

  #set name and parameters
  MLA_name = alg.__class__.__name__
  MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
  MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())


  cv_results = model_selection.cross_validate(alg, X, y, n_jobs= -1,cv = cv_split,scoring='precision',return_train_score=True)
  
  
  # cv_result is a doctionary -> All the results of diff models are saved 
  MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
  MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
  MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
  #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
  MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
  MLA_compare.loc[row_index, 'TrainTestDifference'] = cv_results['train_score'].mean() - cv_results['test_score'].mean()

  row_index += 1 

In [0]:
MLA_compare.sort_values(by = ['TrainTestDifference'], ascending = False, inplace = True)

In [18]:
MLA_compare

,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time,TrainTestDifference
1,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.994817,0.733767,0.0932959,0.00626144,0.26105
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.989699,0.783077,0.101973,0.259905,0.206622
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.793873,0.788714,0.0857072,0.049596,0.00515947
